In [1]:
!pip install qiskit
!pip install qiskit ipywidgets
!pip install qiskit-optimization
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.5/645.5 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 19.8 MB/s eta 0:00:00
  Created wheel for docplex: f

In [11]:
# Imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.primitives import Sampler
from qiskit.circuit.library import TwoLocal
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
from qiskit.circuit import ParameterVector
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qiskit_algorithms.optimizers import COBYLA
import zipfile
import cv2
import seaborn as sns
import os
from PIL import Image
from scipy.spatial.distance import cosine

# Noise settings (if needed)
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

In [5]:
def create_square():
    """
    Generates an 8x8 grayscale image containing a centered square.

    Returns:
        np.array: An 8x8 NumPy array representing a square shape,
                  where pixel values are 1 (white) for the shape and 0 (black) for the background.
    """
    img = np.zeros((8, 8))  # Create an 8x8 black image
    img[2:6, 2:6] = 1  # Fill a 4x4 square in the center
    return img

def create_triangle():
    """
    Generates an 8x8 grayscale image containing a right-angled triangle.

    Returns:
        np.array: An 8x8 NumPy array representing a triangle shape,
                  where pixel values are 1 (white) for the shape and 0 (black) for the background.
    """
    img = np.zeros((8, 8))  # Create an 8x8 black image
    for i in range(4):  # Create a right-angled triangle
        img[6 - i, 2:6 - i] = 1
    return img

def save_images(folder, shape_func, count=100):
    """
    Generates and saves grayscale images of a specific shape.

    Parameters:
        folder (str): Directory where the images will be saved.
        shape_func (function): Function to generate the shape (e.g., create_square or create_triangle).
        count (int): Number of images to generate and save.

    Returns:
        None
    """
    os.makedirs(folder, exist_ok=True)  # Ensure the directory exists
    for i in range(count):
        img_array = shape_func()  # Generate shape
        img = Image.fromarray((img_array * 255).astype(np.uint8), mode='L')  # Convert NumPy array to image
        img.save(f"{folder}/img_{i}.png")  # Save image as PNG

def zip_images(folder, zip_filename):
    """
    Compresses all PNG images in a given folder into a ZIP file.

    Parameters:
        folder (str): Directory containing images to be zipped.
        zip_filename (str): Name of the output ZIP file.

    Returns:
        None
    """
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in os.listdir(folder):
            if file.endswith(".png"):
                zipf.write(os.path.join(folder, file), arcname=file)
    print(f"Zipped {len(os.listdir(folder))} images into {zip_filename}")

# ===========================
# Generate and Zip Images
# ===========================
# UNCOMMENT THIS PART! And adjust the number of images you want.
# Generate and save 100 triangle images
# save_images("triangles", create_triangle, count=100)
# zip_images("triangles", "triangle_images.zip")

# # Generate and save 100 square images
# save_images("squares", create_square, count=100)
# zip_images("squares", "square_images.zip")


Zipped 100 images into triangle_images.zip
Zipped 100 images into square_images.zip


In [6]:
def process_images(zip_path, num_images=None):
    """
    Extracts images from a zip file, converts them to grayscale 8x8,
    flattens them to 1x64 vectors, and normalizes them for quantum processing.

    This function ensures that each image is represented as a valid quantum state
    by normalizing its vector such that the sum of squared values equals 1.

    Args:
        zip_path (str): Path to the ZIP file containing images.
        num_images (int, optional): Number of images to process. If None, all images are processed.

    Returns:
        np.array: An array of shape (num_images, 64) containing normalized image vectors.
    """
    image_vectors = []

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = [file for file in zip_ref.namelist() if file.endswith(('.png', '.jpg', '.jpeg'))]

        if num_images:
            file_list = file_list[:num_images]  # Limit the number of images if specified

        for file in file_list:
            with zip_ref.open(file) as image_file:
                img = Image.open(image_file).convert('L')  # Convert to grayscale
                img = img.resize((8, 8))  # Resize to 8x8 pixels
                img_array = np.array(img).astype(np.float32).flatten()  # Flatten to 1x64 vector

                # Normalize so that sum of squared values = 1 (valid quantum state)
                norm = np.linalg.norm(img_array)
                if norm != 0:
                    img_array /= norm

                image_vectors.append(img_array)

    return np.array(image_vectors)

# ===========================
# Quantum Shape State Functions
# ===========================
def quantum_triangle(zip_path):
    """
    Loads triangle images from a ZIP file and converts them into quantum state vectors.

    Args:
        zip_path (str): Path to the ZIP file containing triangle images.

    Returns:
        np.array: An array of shape (num_images, 64) representing triangle images as quantum states.
    """
    return process_images(zip_path)

def quantum_square(zip_path):
    """
    Loads square images from a ZIP file and converts them into quantum state vectors.

    Args:
        zip_path (str): Path to the ZIP file containing square images.

    Returns:
        np.array: An array of shape (num_images, 64) representing square images as quantum states.
    """
    return process_images(zip_path)


In [7]:
def amplitude_encode(amplitudes):
    """
    Normalize the input data and encode it into a quantum state.

    Parameters:
        amplitudes (list or np.array): Input data to be encoded.

    Returns:
        QuantumCircuit: Quantum circuit with the encoded data.
    """
    amplitudes = np.array(amplitudes)
    # Check that the size is a power of 2
    if amplitudes.size & (amplitudes.size - 1) != 0:
        raise ValueError("Input size must be a power of 2.")
    # Normalize the vector
    amplitudes = amplitudes / np.sqrt(np.sum(amplitudes**2))
    num_qubits = int(np.log2(amplitudes.size))
    qc = QuantumCircuit(num_qubits)
    qc.initialize(amplitudes, range(num_qubits))
    return qc

In [8]:
def u_theta(amplitudes, parameters, entanglement, layers):
    """
    Create a quantum neural network (QNN) circuit.
    This circuit encodes the input amplitudes and then applies a parametrized TwoLocal circuit.
    Parameters:
        amplitudes (list or np.array): Input data to be encoded into quantum states.
        parameters (list or np.array): Trainable parameters for the TwoLocal circuit.
        entanglement (str or list): Defines the entanglement pattern between qubits
                                    (e.g., 'linear', 'full', or a custom list).
        layers (int): Number of repetitions (depth) of the TwoLocal circuit.

    Returns:
        QuantumCircuit: A quantum circuit with input encoding and trainable variational layers.

    """
    num_qubits = int(np.log2(len(amplitudes)))

    # Create the parametrized circuit using TwoLocal
    twolocal = TwoLocal(
        num_qubits=num_qubits,
        rotation_blocks=['ry'],
        entanglement_blocks='cx',
        entanglement=entanglement,
        reps=layers  # number of hidden layers
    )

    # Bind parameters to the circuit
    param_dict = dict(zip(twolocal.parameters, parameters))
    twolocal.assign_parameters(param_dict, inplace=True)

    # Compose the feature map and the parametrized circuit
    qnn = QuantumCircuit(num_qubits, num_qubits)
    feature_map = amplitude_encode(amplitudes)
    qnn.compose(feature_map, inplace=True)
    qnn.compose(twolocal, inplace=True)
    # qnn.measure(range(num_qubits), range(num_qubits)) # COMMENT OUT IF USING SAMPLER!

    return qnn

In [12]:
def cost_function(params, features, labels, entanglement, layers):
    """
    Compute the total cost (sum of squared differences) between the measured output
    and the one-hot encoded label over all training examples.
    """
    total_error = 0
    similarity_sum = 0
    for feature, label in zip(features, labels):
        target_state = np.array(label)  # Ensure label is an array
        feature = np.array(feature)     # Ensure feature is an array

        # Apply quantum circuit to input feature
        qnn = u_theta(feature, params, entanglement, layers)

        # Simulate the circuit using Statevector
        statevector = Statevector.from_instruction(qnn)

        # Get the probabilities from the statevector data
        probabilities = np.abs(statevector.data) ** 2  # Square the amplitudes to get probabilities
        output_state = np.array(probabilities)

        # Compute squared error between output and target
        total_error += np.sum((output_state - target_state) ** 2)

        # Cosine similarity (1 means identical, 0 means orthogonal)
        similarity = 1 - cosine(output_state, target_state)
        similarity_sum += similarity

    avg_similarity = similarity_sum / len(features)

    print(f"Feature: {feature}, Label: {label}, Probabilities: {probabilities}, Error: {total_error/len(features)}, Avg Similarity: {avg_similarity:.4f}")

    return total_error / len(features), avg_similarity

In [18]:
def optimization(costfunction, params, features, labels, optimizer, entanglement, layers, color):
    """
    Perform optimization to minimize the cost function for a quantum machine learning model.

    Parameters:
        costfunction (function): The cost function to be minimized. This can use either
                                 statevector simulation or a sampler for probability extraction.
        params (array): Initial parameter values for the variational quantum circuit.
        features (array): The input feature vectors (e.g., quantum-encoded image data).
        labels (array): The target output states (e.g., quantum state representations of transformed images).
        optimizer (str): The optimization algorithm to use. Examples include:
                         - "COBYLA" (Constrained Optimization By Linear Approximations)
                         - "L-BFGS-B" (Limited-memory Broyden–Fletcher–Goldfarb–Shanno with Box constraints)
                         - "SLSQP" (Sequential Least Squares Programming)
        entanglement (str): The entanglement strategy for the variational circuit (e.g., "linear", "full").
        layers (int): Number of layers (repetitions) in the variational quantum circuit.
        color (str): Color for the optimization progress plot (e.g., "blue", "red").

    Returns:
        dict: A dictionary containing:
              - "optimized_params" (array): The optimized parameter values.
              - "final_cost" (float): The final cost function value after optimization.
              - "cost_history" (list): A history of cost values throughout the optimization process.

    The function also plots the cost function's progress over optimization iterations.

    """
    cost_history = []
    accuracy_history = []
    def callback_function(params):
        """
        Track the cost value during optimization.
        """
        cost, accuracy = costfunction(params, features, labels, entanglement, layers)
        cost_history.append(cost)
        accuracy_history.append(accuracy)
        print(f"Iteration {len(cost_history)} | Cost: {cost:.4f} | Accuracy: {accuracy * 100:.2f}%")

    result = minimize(
        fun=lambda p, f, l, e, ly: costfunction(p, f, l, e, ly)[0],
        x0=params,
        args=(features, labels, entanglement, layers),
        method=optimizer,  # You can experiment with other methods like COBYLA or SLSQP
        callback=callback_function,
        options={"disp": True, "maxiter": 200}
    )
     # Plot Cost & Accuracy
    fig, ax1 = plt.subplots(figsize=(8, 5))

    ax1.plot(cost_history, marker="o", color=color, label="Cost")
    ax1.set_xlabel("Iteration")
    ax1.set_ylabel("Cost Function", color=color)
    ax1.tick_params(axis="y", labelcolor=color)

    ax2 = ax1.twinx()  # Create secondary y-axis
    ax2.plot(accuracy_history, marker="s", color="steelblue", label="Accuracy")
    ax2.set_ylabel("Accuracy", color="steelblue")
    ax2.tick_params(axis="y", labelcolor="steelblue")

    plt.title("Optimization Progress: Cost & Accuracy")
    fig.tight_layout()
    plt.grid()
    plt.show()

    return {
        "optimized_params": result.x,
        "final_cost": result.fun,
        "cost_history": cost_history,
        "accuracy_history": accuracy_history,
    }

In [21]:
# TESTING!
# Uncomment the following code to train the model.

# X_train, X_test, y_train, y_test = train_test_split(
#     process_images("shapes.zip")[0],  # Extract and preprocess features
#     process_images("shapes.zip")[1],  # Extract corresponding target quantum states
#     test_size=0.2,  # 20% of the data is reserved for testing
#     random_state=42  # Ensures reproducibility of the split
# )

# print("Training features shape:", X_train.shape, "Training labels shape:", y_train.shape)

# # Initialize parameters. The number of parameters must match the number of parameters in TwoLocal.
# # In this example, we initialize with 64 parameters (adjust if needed based on the circuit size).
# initial_params = np.random.uniform(-np.pi, np.pi, 64)

# # Run the optimization process using the specified optimizer and circuit configuration.
# optimization(
#     cost_function,  # The cost function to minimize
#     initial_params,  # Initial variational parameters
#     X_train,  # Training feature set
#     y_train,  # Training labels (quantum states)
#     "L-BFGS-B",  # Chosen classical optimizer
#     "full",  # Entanglement strategy
#     5,  # Number of layers in the variational quantum circuit
#     "#FF1493"  # Color for the optimization plot
# )
